Воспроизводимый текст представлен ниже

In [ ]:
INPUT_TEXT = """
The desert planet holds ancient secrets, buried beneath endless dunes and silence.
Water is life—and on Arrakis, every drop is measured, honored, and fiercely protected.
A leader must see not only the present danger, but the shadow of consequence stretching far into the future.
The spice flows through all things: it shapes fate, awakens prescience, and binds the universe together.
To know the desert is to understand patience, adaptation, and the quiet power of survival.
"""

# Библиотеки

In [ ]:
!pip install gtts -qq
!pip install torchcodec -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/sv9-ru/TTS-System.git

Cloning into 'TTS-System'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 2.77 MiB | 6.48 MiB/s, done.


In [ ]:
from gtts import gTTS
from IPython.display import Audio, display
import os
from transformers import VitsModel, AutoTokenizer
import torch
import torchaudio
import scipy.io.wavfile
import numpy as np
import wave

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.2 MB/s eta 0:00:00


In [ ]:
zvukogram = "/content/TTS-System/abbi.mp3"

# Генерация Голоса

In [ ]:
# 2. Имя выходного MP3 файла
OUTPUT_FILENAME = "gtts_api_output.mp3"

# 3. Язык (по умолчанию 'en' - английский)
LANGUAGE = 'en'

# --- Генерация ---

try:
    # Создаем объект gTTS
    tts = gTTS(text=INPUT_TEXT, lang=LANGUAGE, slow=False)

    # Сохраняем аудио в файл
    tts.save(OUTPUT_FILENAME)

    print(f"✅ Голос успешно сгенерирован и сохранен как '{OUTPUT_FILENAME}'")

    # --- Воспроизведение в Colab ---

except Exception as e:
    print(f"❌ Произошла ошибка при генерации голоса: {e}")

✅ Голос успешно сгенерирован и сохранен как 'gtts_api_output.mp3'


In [ ]:
print("ЗАГРУЗКА НЕЙРОСЕТЕВОЙ TTS МОДЕЛИ ИЗ HUGGING FACE ===")

try:


    # Проверяем доступность GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f" Используемое устройство: {device}")

    print("Загрузка модели Microsoft SpeechT5...")

    # Используем VITS модель - современную нейросетевую архитектуру
    model_name = "facebook/mms-tts-eng"  # Мультиязычная модель от Meta

    # Загружаем модель и токенизатор
    model = VitsModel.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    print("Нейросетевая модель успешно загружена!")
    print(f"Модель: {model_name}")
    print(f"Архитектура: VITS (Variational Inference with adversarial learning for end-to-end Text-to-Speech)")

except Exception as e:
    print(f"❌ Ошибка при загрузке модели: {e}")
    model = None
    tokenizer = None

ЗАГРУЗКА НЕЙРОСЕТЕВОЙ TTS МОДЕЛИ ИЗ HUGGING FACE ===
 Используемое устройство: cuda
Загрузка модели Microsoft SpeechT5...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

Нейросетевая модель успешно загружена!
Модель: facebook/mms-tts-eng
Архитектура: VITS (Variational Inference with adversarial learning for end-to-end Text-to-Speech)


In [ ]:
# Создание аудио с помощью нейросетевой модели
print("\n СОЗДАНИЕ АУДИО С ПОМОЩЬЮ НЕЙРОСЕТИ ===")

if model is not None and tokenizer is not None:
    try:


        # Текст для синтеза (убедитесь, что он соответствует вашему заданию)

        device = "cuda" if torch.cuda.is_available() else "cpu"

        print("Текст для синтеза:")
        print(INPUT_TEXT.strip())

        # Токенизация текста
        inputs = tokenizer(INPUT_TEXT, return_tensors="pt").to(device)

        print("Генерация речи нейросетью...")

        # Генерация аудио с помощью модели
        with torch.no_grad():
            output = model(**inputs)
            waveform = output.waveform[0].cpu().numpy()

        # Нормализация аудио
        waveform = waveform / np.max(np.abs(waveform))

        # Сохранение аудио
        neural_output = "neural_tts_hf.wav"
        sample_rate = model.config.sampling_rate

        # Сохраняем в WAV файл
        torchaudio.save(neural_output, torch.tensor(waveform).unsqueeze(0), sample_rate)

        print(f"✅ Нейросетевое аудио создано: {neural_output}")

        if os.path.exists(neural_output):
            file_size = os.path.getsize(neural_output) / 1024

            # Открываем файл с помощью стандартного модуля wave
            with wave.open(neural_output, 'rb') as wf:
                num_frames = wf.getnframes()
                sample_rate_wav = wf.getframerate()

            duration = num_frames / sample_rate_wav

            print(f"Характеристики аудио:")
            print(f"   - Длительность: {duration:.1f} сек")
            print(f"   - Размер: {file_size:.1f} KB")
            print(f"   - Частота: {sample_rate_wav} Hz")

            # Воспроизводим результат
            print("Воспроизведение результата:")


        else:
            print("❌ Файл не был создан")

    except Exception as e:
        print(f"❌ Ошибка при генерации аудио: {e}")

else:
    print("❌ Нейросетевая модель недоступна. Убедитесь, что Шаг 1 (загрузка модели) был выполнен успешно.")


 СОЗДАНИЕ АУДИО С ПОМОЩЬЮ НЕЙРОСЕТИ ===
Текст для синтеза:
The desert planet holds ancient secrets, buried beneath endless dunes and silence.
Water is life—and on Arrakis, every drop is measured, honored, and fiercely protected.
A leader must see not only the present danger, but the shadow of consequence stretching far into the future.
The spice flows through all things: it shapes fate, awakens prescience, and binds the universe together.
To know the desert is to understand patience, adaptation, and the quiet power of survival.
Генерация речи нейросетью...
✅ Нейросетевое аудио создано: neural_tts_hf.wav
Характеристики аудио:
   - Длительность: 31.9 сек
   - Размер: 997.6 KB
   - Частота: 16000 Hz
Воспроизведение результата:


# Результат

In [ ]:
print("сгенерированый при помощи tortoise")
display(Audio(OUTPUT_FILENAME))
print("="*50)
print("сгенерированый при помощи нейосети ИЗ HUGGING FACE")
display(Audio(neural_output))
print("="*50)
print("сгенерированый в zvukogram")
display(Audio(zvukogram))

сгенерированый при помощи tortoise


сгенерированый при помощи нейосети ИЗ HUGGING FACE


сгенерированый в zvukogram
